In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/Asobi

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/Asobi


In [16]:
from google.colab.patches import cv2_imshow


import cv2
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
import numpy as np

def number_maker(element, count):
  ttfontname = "./font/Helvetica.ttc"
  fontsize = 100
  text = str(element[0])

  # 画像サイズ，背景色，フォントの色を設定
  canvasSize    = (200, 200)
  RGB = (element[1],element[2],element[3])
  alpha = 0.1
  backgroundRGB = (int(element[1]*alpha),int(element[2]*alpha),int(element[3]*alpha))
  # 文字を描く画像の作成
  img  = PIL.Image.new('RGB', canvasSize, backgroundRGB)
  draw = PIL.ImageDraw.Draw(img)

  # 用意した画像に文字列を描く
  font = PIL.ImageFont.truetype(ttfontname, fontsize)
  textWidth, textHeight = draw.textsize(text,font=font)
  textTopLeft = (canvasSize[0]//6, canvasSize[1]//2-textHeight//2)
  draw.text(textTopLeft, text, fill=RGB, font=font)
  img = np.asarray(img)
  return img

# 平均明度を計算する。入力：ndarray型, 出力：int型
def average_brightness(img):
    return int(img.mean())

# 平均彩度を計算する。入力：ndarray型, 出力：int型 × 3
def average_saturation(img):
    r = int(img[:, :, 0].mean())
    g = int(img[:, :, 1].mean())
    b = int(img[:, :, 2].mean())
    return r, g, b


# メイン関数
def main(motogazo, x_bunkatsu, y_bunkatsu, x_size_result, y_size_result):
    # 元画像を読み込む
    img = cv2.imread(motogazo)

    # サブ画像のサイズを計算する
    x_size = int(img.shape[0]/x_bunkatsu)
    y_size = int(img.shape[1]/y_bunkatsu)

    # 元画像の明度、彩度を計算してListに格納する。
    img_elements = []
    for y in range(y_bunkatsu):
        for x in range(x_bunkatsu):
            tmp = img[x*x_size:(x+1)*x_size, y*y_size:(y+1)*y_size]
            brightness = average_brightness(tmp)
            r, g, b = average_saturation(tmp)
            img_elements.append(np.array([brightness, r, g, b]))
    

    print("Mozaiku art image size: {}".format(len(img_elements)))

    files = []
    count = 0
    for element in img_elements:
      files.append(number_maker(element,count))
      count += 1

    # モザイクアートの下地を生成する。
    img_result = np.zeros((x_size_result, y_size_result, 3), dtype=np.uint8)

    x_size = int(x_size_result/x_bunkatsu)
    y_size = int(y_size_result/y_bunkatsu)


    for y in range(y_bunkatsu):
        for x in range(x_bunkatsu):
            img = files[x+x_bunkatsu*y]
            img = cv2.resize(img, (x_size, y_size))
            img_result[x*x_size:(x+1)*x_size, y*y_size:(y+1)*y_size] = img

    cv2_imshow(img_result)
    print("Done")
    return img_result


if __name__ == "__main__":
    """
    CONSTRAINTS:
        1.motogazo size is the multiple of x_bunkatsu and y_bunkatsu
        2.x(y)_size_result is the multiple of x(y)_bunkatsu
    """
    # 入力画像の情報
    motogazo = './src/pearl.png'
    x_bunkatsu = 13*10*2
    y_bunkatsu = 11*10*2

    # 出力画像の情報
    result_filename = './mozaiku.png'
    x_size_result = 1430*4
    y_size_result = 1210*4

    img_result = main(motogazo, x_bunkatsu, y_bunkatsu, x_size_result, y_size_result)

    cv2.imwrite(result_filename, img_result)

Output hidden; open in https://colab.research.google.com to view.